Import Packages

In [1]:
# Install required packages
# Run this cell first before executing the rest of the notebook

!pip install \
  langchain==0.3.27 \
  langchain-core==0.3.79 \
  langchain-openai==0.3.11 \
  langchain-community==0.3.31 \
  langgraph==0.3.7 \
  tavily-python \
  yfinance==0.2.66 \
  chromadb==1.3.4 \
  pypdf==6.2.0 \
  tiktoken==0.12.0

print("✅ All packages installed successfully!")

Defaulting to user installation because normal site-packages is not writeable
  Using cached langchain-0.3.27-py3-none-any.whl.metadata (7.8 kB)
  Using cached langchain_community-0.3.31-py3-none-any.whl.metadata (3.0 kB)
  Using cached langchain_text_splitters-0.3.11-py3-none-any.whl.metadata (1.8 kB)
Using cached langchain-0.3.27-py3-none-any.whl (1.0 MB)
Using cached langchain_community-0.3.31-py3-none-any.whl (2.5 MB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.2/19.2 MB 5.7 MB/s  0:00:03m0:00:0100:01
Using cached langchain_text_splitters-0.3.11-py3-none-any.whl (33 kB)
  Attempting uninstall: pypdf
    Found existing installation: pypdf 5.4.0
    Uninstalling pypdf-5.4.0:
      Successfully uninstalled pypdf-5.4.0
  Attempting uninstall: langchain-core━━━━━━━━━━━━━━━━━━━━━━━━━━━━  3/13 [langgraph-sdk]
    Found existing installation: langchain-core 0.3.65━━━━━━━━  3/13 [langgraph-sdk]
    Uninstalling langchain-core-0.3.65:━━━━━━━━━━━━━━━━━━━━━━━  3/13 [langgraph-sdk]
      Suc

In [14]:
# Core imports
import os
import sys
import json
import logging
from typing import Dict, List, Literal
from datetime import datetime

# LangChain imports
from langchain_core.messages import SystemMessage, HumanMessage, ToolMessage
from langchain_openai import ChatOpenAI
from langchain_core.tools import tool
from langgraph.graph import StateGraph, END, MessagesState
from langgraph.checkpoint.memory import MemorySaver
from langgraph.prebuilt import ToolNode
from langgraph.graph.message import add_messages
from typing import Annotated, Sequence, TypedDict

# External tools
import yfinance as yf

# Configure logging to see agent's decision-making process
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s [%(levelname)s] %(message)s',
    datefmt='%H:%M:%S'
)
logger = logging.getLogger(__name__)

print("✅ All imports successful!")

✅ All imports successful!


Config.json

In [5]:
import json
# Load the JSON file and extract values
file_name = 'config.json'
with open(file_name, 'r') as file:
    config = json.load(file)


os.environ["OPENAI_API_KEY"] = config.get("API_KEY")
os.environ["OPENAI_API_BASE"] = config.get("OPENAI_API_BASE")
os.environ["TAVILY_API_KEY"] = config.get("TAVILY_API_KEY")

print("✅ Configuration loaded successfully!")
print(f"   Using API base: {os.environ['OPENAI_API_BASE']}")

✅ Configuration loaded successfully!
   Using API base: https://aibe.mygreatlearning.com/openai/v1


Traditional Prompt

In [19]:
TRADITIONAL_PROMPT = """You are a helpful assistant.
Answer the user's question about stock information."""

print("📋 Traditional LLM Prompt:")
print("="*80)
print(TRADITIONAL_PROMPT)
print("="*80)
print("\n❌ Problems with this approach:")
print("   • No initiative - waits for user to specify what they want")
print("   • No comprehensive analysis - just answers the question")
print("   • Asks follow-up questions instead of taking action")
print("   • Provides minimal information")

📋 Traditional LLM Prompt:
You are a helpful assistant.
Answer the user's question about stock information.

❌ Problems with this approach:
   • No initiative - waits for user to specify what they want
   • No comprehensive analysis - just answers the question
   • Asks follow-up questions instead of taking action
   • Provides minimal information


Agent Charter Basic

In [18]:
AGENT_CHARTER_BASIC = """You are an autonomous Financial Research Analyst specializing in AI-focused companies.

YOUR PRIMARY GOAL:
Generate a comprehensive financial analysis report for the requested company that includes:
1. Current stock price and 3-year performance trends
2. Recent financial news and market sentiment
3. Key risks and opportunities
4. Investment recommendation with supporting evidence

Take initiative to gather all necessary information to achieve this goal.
Don't just answer questions - proactively provide complete, actionable insights."""

print("📋 Agent Charter (Goal-Oriented):")
print("="*80)
print(AGENT_CHARTER_BASIC)
print("="*80)
print("\n✅ Benefits of this approach:")
print("   • Defines a clear mission and goal")
print("   • Specifies expected output format")
print("   • Encourages proactive information gathering")
print("   • Focuses on actionable insights, not just answers")

📋 Agent Charter (Goal-Oriented):
You are an autonomous Financial Research Analyst specializing in AI-focused companies.

YOUR PRIMARY GOAL:
Generate a comprehensive financial analysis report for the requested company that includes:
1. Current stock price and 3-year performance trends
2. Recent financial news and market sentiment
3. Key risks and opportunities
4. Investment recommendation with supporting evidence

Take initiative to gather all necessary information to achieve this goal.
Don't just answer questions - proactively provide complete, actionable insights.

✅ Benefits of this approach:
   • Defines a clear mission and goal
   • Specifies expected output format
   • Encourages proactive information gathering
   • Focuses on actionable insights, not just answers


Tool 1: Stock Price Tool

In [7]:
@tool
def get_stock_price(ticker: str) -> Dict:
    """
    Returns the current stock price and basic information for a given ticker symbol.

    This tool fetches real-time stock data including current price, day's range,
    volume, and market cap. Use this when you need current stock pricing information.

    Args:
        ticker: Stock ticker symbol (e.g., 'AAPL', 'MSFT', 'GOOGL')

    Returns:
        dict: {
            'ticker': str,
            'current_price': float,
            'currency': str,
            'day_high': float,
            'day_low': float,
            'volume': int,
            'market_cap': int,
            'timestamp': str,
            'status': str,
            'error': str (optional)
        }

    Example:
        >>> result = get_stock_price("AAPL")
        >>> print(f"Apple stock price: ${result['current_price']}")
    """
    try:
        stock = yf.Ticker(ticker.upper())
        info = stock.info

        # Get current price (try multiple fields as yfinance API can vary)
        current_price = (
            info.get('currentPrice') or
            info.get('regularMarketPrice') or
            info.get('previousClose')
        )

        if current_price is None:
            return {
                'ticker': ticker.upper(),
                'status': 'error',
                'error': f'Could not retrieve price data for {ticker}. Ticker may be invalid.'
            }

        result = {
            'ticker': ticker.upper(),
            'current_price': round(current_price, 2),
            'currency': info.get('currency', 'USD'),
            'day_high': info.get('dayHigh', info.get('regularMarketDayHigh')),
            'day_low': info.get('dayLow', info.get('regularMarketDayLow')),
            'volume': info.get('volume', info.get('regularMarketVolume')),
            'market_cap': info.get('marketCap'),
            'company_name': info.get('longName', info.get('shortName')),
            'timestamp': datetime.now().isoformat(),
            'status': 'success'
        }

        return result

    except Exception as e:
        return {
            'ticker': ticker.upper(),
            'status': 'error',
            'error': f'Error fetching stock data: {str(e)}',
            'timestamp': datetime.now().isoformat()
        }

print("✅ Tool 1: get_stock_price() - Defined")
print("   Purpose: Fetch real-time stock price and basic metrics")
print("   Data Source: Yahoo Finance (yfinance)")

✅ Tool 1: get_stock_price() - Defined
   Purpose: Fetch real-time stock price and basic metrics
   Data Source: Yahoo Finance (yfinance)


Tool 2: Stock History Tool

In [16]:
@tool
def get_stock_history(ticker: str, period: str = "1y") -> Dict:
    """
    Returns historical stock price data for analysis of 3-year performance.

    This tool fetches historical stock data over a specified period, useful for
    analyzing trends, calculating returns, and assessing long-term performance.

    Args:
        ticker: Stock ticker symbol (e.g., 'AAPL', 'MSFT', 'GOOGL')
        period: Time period for historical data. Options: '1mo', '3mo', '6mo',
                '1y', '2y', '3y', '5y', '10y'. Default is '1y'.

    Returns:
        dict: {
            'ticker': str,
            'period': str,
            'start_date': str,
            'end_date': str,
            'start_price': float,
            'end_price': float,
            'return_pct': float,
            'high': float,
            'low': float,
            'avg_volume': int,
            'data_points': int,
            'status': str
        }
    """
    try:
        stock = yf.Ticker(ticker.upper())
        hist = stock.history(period=period)

        if hist.empty:
            return {
                'ticker': ticker.upper(),
                'status': 'error',
                'error': f'No historical data available for {ticker} over period {period}'
            }

        # Calculate key metrics
        start_price = hist['Close'].iloc[0]
        end_price = hist['Close'].iloc[-1]
        return_pct = ((end_price - start_price) / start_price) * 100

        result = {
            'ticker': ticker.upper(),
            'period': period,
            'start_date': hist.index[0].strftime('%Y-%m-%d'),
            'end_date': hist.index[-1].strftime('%Y-%m-%d'),
            'start_price': round(start_price, 2),
            'end_price': round(end_price, 2),
            'return_pct': round(return_pct, 2),
            'high': round(hist['High'].max(), 2),
            'low': round(hist['Low'].min(), 2),
            'avg_volume': int(hist['Volume'].mean()),
            'data_points': len(hist),
            'status': 'success'
        }

        return result

    except Exception as e:
        return {
            'ticker': ticker.upper(),
            'status': 'error',
            'error': f'Error fetching historical data: {str(e)}'
        }

print("✅ Tool 2: get_stock_history() - Defined")
print("   Purpose: Fetch historical performance for trend analysis")
print("   Key Metric: 3-year return percentage")

✅ Tool 2: get_stock_history() - Defined
   Purpose: Fetch historical performance for trend analysis
   Key Metric: 3-year return percentage


In [9]:
from langchain_community.tools.tavily_search import TavilySearchResults

# Initialize Tavily search tool
tavily_tool = TavilySearchResults(
    max_results=5,
    search_depth="advanced",
    include_answer=True,
    include_raw_content=False,
    include_images=False,
)

@tool
def search_financial_news(query: str) -> List[Dict]:
    """
    Searches real-time financial news using Tavily search API.

    This tool searches the web for recent financial news articles related to your query.
    Use this to find market sentiment, recent developments, and news about companies.

    Args:
        query: Search query string (e.g., "Apple AI initiatives 2024")

    Returns:
        list: List of news articles with:
            - title: Article title
            - url: Article URL
            - content: Article snippet/summary
            - score: Relevance score

    Example:
        >>> results = search_financial_news("Microsoft AI research")
        >>> for article in results:
        >>>     print(f"{article['title']}: {article['url']}")
    """
    try:
        results = tavily_tool.invoke({"query": query})
        return results
    except Exception as e:
        return [{
            'status': 'error',
            'error': f'Error searching news: {str(e)}'
        }]

print("✅ Tool 3: search_financial_news() - Defined")
print("   Purpose: Search real-time financial news")
print("   Data Source: Tavily Search API")

✅ Tool 3: search_financial_news() - Defined
   Purpose: Search real-time financial news
   Data Source: Tavily Search API


/var/folders/dc/flyc_kxs2d554dt8_szth1_40000gn/T/ipykernel_9329/2699283982.py:4: LangChainDeprecationWarning: The class `TavilySearchResults` was deprecated in LangChain 0.3.25 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-tavily package and should be used instead. To use it run `pip install -U :class:`~langchain-tavily` and import as `from :class:`~langchain_tavily import TavilySearch``.
  tavily_tool = TavilySearchResults(


Tool 3: Financial News Search Tool

In [10]:
@tool
def analyze_sentiment(text: str) -> Dict:
    """
    Analyzes the sentiment of financial text using OpenAI.

    This tool analyzes the sentiment (positive/negative/neutral) of news articles,
    reports, or any financial text. Returns a sentiment label and confidence score.

    Args:
        text: Text to analyze (article, headline, report excerpt)

    Returns:
        dict: {
            'sentiment': str ('positive', 'negative', or 'neutral'),
            'score': float (0.0 to 1.0, where 1.0 is most positive),
            'confidence': float (0.0 to 1.0),
            'reasoning': str (brief explanation)
        }

    Example:
        >>> result = analyze_sentiment("Apple reports record earnings...")
        >>> print(f"Sentiment: {result['sentiment']} (score: {result['score']})")
    """
    try:
        model = ChatOpenAI(model="gpt-4o-mini", temperature=0)

        prompt = f"""Analyze the sentiment of this financial text and provide:
1. Sentiment label: positive, negative, or neutral
2. Score: 0.0 (very negative) to 1.0 (very positive), 0.5 is neutral
3. Confidence: 0.0 to 1.0
4. Brief reasoning

Text: {text}

Respond in JSON format:
{{
    "sentiment": "positive|negative|neutral",
    "score": 0.0-1.0,
    "confidence": 0.0-1.0,
    "reasoning": "brief explanation"
}}"""

        response = model.invoke(prompt) # Ask the llm to predict the sentiment of the given text
        result = json.loads(response.content)
        result['status'] = 'success'
        return result

    except Exception as e:
        # Fallback to simple sentiment if OpenAI fails
        positive_words = ['growth', 'profit', 'gain', 'success', 'up', 'positive', 'strong']
        negative_words = ['loss', 'decline', 'down', 'weak', 'risk', 'concern', 'negative']

        text_lower = text.lower()
        pos_count = sum(1 for word in positive_words if word in text_lower)
        neg_count = sum(1 for word in negative_words if word in text_lower)

        if pos_count > neg_count:
            sentiment = 'positive'
            score = 0.6 + (pos_count * 0.05)
        elif neg_count > pos_count:
            sentiment = 'negative'
            score = 0.4 - (neg_count * 0.05)
        else:
            sentiment = 'neutral'
            score = 0.5

        return {
            'sentiment': sentiment,
            'score': max(0.0, min(1.0, score)),
            'confidence': 0.6,
            'reasoning': 'Fallback keyword-based analysis',
            'status': 'success (fallback)',
            'note': f'OpenAI analysis failed: {str(e)}'
        }

print("✅ Tool 4: analyze_sentiment() - Defined")
print("   Purpose: Analyze sentiment of financial text")
print("   Method: OpenAI gpt-4o-mini with keyword fallback")

✅ Tool 4: analyze_sentiment() - Defined
   Purpose: Analyze sentiment of financial text
   Method: OpenAI gpt-4o-mini with keyword fallback


Tool 4: Sentiment Analysis Tool

In [12]:
# Test get_stock_price manually
print("🧪 Testing get_stock_price tool with ticker 'AAPL':\n")
result = get_stock_price("AAPL")
print(json.dumps(result, indent=2))

print("\n" + "="*80 + "\n")

# Test get_stock_history manually
print("🧪 Testing get_stock_history tool with ticker 'AAPL' and period '3y':\n")
result = get_stock_history.invoke({"ticker": "AAPL", "period": "3y"})
print(json.dumps(result, indent=2))

🧪 Testing get_stock_price tool with ticker 'AAPL':

{
  "ticker": "AAPL",
  "current_price": 260.45,
  "currency": "USD",
  "day_high": 261.38,
  "day_low": 255.54,
  "volume": 16923985,
  "market_cap": 3828148076544,
  "company_name": "Apple Inc.",
  "timestamp": "2026-02-17T11:45:55.144613",
  "status": "success"
}


🧪 Testing get_stock_history tool with ticker 'AAPL' and period '3y':

{
  "ticker": "AAPL",
  "period": "3y",
  "start_date": "2023-02-17",
  "end_date": "2026-02-17",
  "start_price": 150.37,
  "end_price": 260.45,
  "return_pct": 73.21,
  "high": 288.35,
  "low": 141.85,
  "avg_volume": 55946451,
  "data_points": 751,
  "status": "success"
}


Full Agent Charter

In [17]:
AGENT_CHARTER_FULL = """You are an autonomous Financial Research Analyst Agent specializing in AI sector investments.

════════════════════════════════════════════════════════════════════════════════
PRIMARY MISSION
════════════════════════════════════════════════════════════════════════════════

Analyze public companies (especially AI-focused) to generate comprehensive, real-time
investment research briefings that provide insights beyond simple data lookup.

TARGET OUTPUT:
A structured report covering:
• Financial Health: Stock performance, 3-year trends, key metrics
• Market Sentiment: News analysis with sentiment scores
• AI Research Activity: Current AI projects and innovations
• Risk Assessment: Key risks and opportunities
• Investment Recommendation: Data-driven rating with confidence level

════════════════════════════════════════════════════════════════════════════════
AVAILABLE TOOLS
════════════════════════════════════════════════════════════════════════════════

Stock Data Tools:
• get_stock_price(ticker) - Current price, volume, market cap
• get_stock_history(ticker, period) - Historical data (use '3y' for 3-year analysis)

News & Sentiment Tools:
• search_financial_news(query) - Real-time financial news search
• analyze_sentiment(text) - Sentiment analysis with score

════════════════════════════════════════════════════════════════════════════════
PROACTIVE BEHAVIOR - Take Initiative
════════════════════════════════════════════════════════════════════════════════

✓ ALWAYS gather comprehensive data, not just what's explicitly requested
✓ ALWAYS check 3-year historical performance, not just current price
✓ ALWAYS analyze recent news sentiment, even if not asked
✓ ALWAYS identify risks proactively, don't wait to be asked
✓ ALWAYS make a clear recommendation with confidence level

✗ NEVER stop at surface-level data
✗ NEVER provide analysis without supporting evidence
✗ NEVER ignore warning signs in the data

════════════════════════════════════════════════════════════════════════════════
REACTIVE BEHAVIOR - Error Handling & Adaptability
════════════════════════════════════════════════════════════════════════════════

When Tools Fail:
• If a tool returns an error, IMMEDIATELY try an alternative approach
• If stock data fails, explain the limitation and use news/company info instead
• If news search fails, note this gap and continue with available data
• NEVER stop your analysis due to a single tool failure
• Log all errors but maintain momentum toward your goal

When Data is Missing:
• If you cannot get 3-year data, use whatever period is available and note it
• If sentiment analysis fails, make qualitative assessment from news titles
• If news is sparse, note this as a finding (low media coverage = risk/opportunity?)
• ALWAYS work with what you have, document what you don't have

════════════════════════════════════════════════════════════════════════════════
AUTONOMOUS BEHAVIOR - Independence & Judgment
════════════════════════════════════════════════════════════════════════════════

Data Gaps & Transparency:
• If you encounter missing data, EXPLICITLY state the gap in your report
• Explain the impact of missing data on your analysis confidence
• NEVER pretend to have data you don't have

Source Citation (MANDATORY):
• You MUST cite the source for every factual claim
• Include timestamps for time-sensitive data (stock prices, news)
• Format: [Source: tool_name, timestamp]

Example:
✓ "AAPL is trading at $178.45 [Source: get_stock_price, 2024-10-30 13:30]"
✓ "Recent news shows positive sentiment (score: 0.75) [Source: analyze_sentiment]"
✗ "The stock is doing well" (no source, no metrics)

Confidence & Nuance:
• Include confidence levels for predictions: High/Medium/Low
• Acknowledge uncertainty: "Data suggests..." vs "Data confirms..."
• Note when analysis is limited by data availability

════════════════════════════════════════════════════════════════════════════════
QUALITY STANDARDS
════════════════════════════════════════════════════════════════════════════════

Every Report Must Include:
1. Executive Summary (2-3 sentences)
2. Financial Metrics (with sources and timestamps)
3. Sentiment Analysis (with scores and article count)
4. Risk Factors (minimum 2-3 identified)
5. AI Research Activity (verified presence/absence)
6. Recommendation (Buy/Hold/Sell with confidence %)
7. Source Citations (for all claims)
8. Gaps & Limitations (what data was unavailable)

Remember: You are AUTONOMOUS. Take initiative, handle errors gracefully, and
always drive toward your goal of comprehensive investment analysis.
"""

print("📋 Full Agent Charter with Constraints:")
print("="*80)
print("Key features:")
print("✅ Proactive Behavior: Takes initiative, gathers comprehensive data")
print("✅ Reactive Behavior: Handles tool failures, adapts to missing data")
print("✅ Autonomous Behavior: Makes independent judgments, cites sources")
print("✅ Quality Standards: Structured output with confidence levels")
print("="*80)

📋 Full Agent Charter with Constraints:
Key features:
✅ Proactive Behavior: Takes initiative, gathers comprehensive data
✅ Reactive Behavior: Handles tool failures, adapts to missing data
✅ Autonomous Behavior: Makes independent judgments, cites sources
✅ Quality Standards: Structured output with confidence levels


Agent State Definition

In [15]:
class SimpleAgentState(MessagesState):
    """
    State for the financial research agent.
    Tracks the conversation history with message accumulation.
    """
    messages: Annotated[Sequence, add_messages]

print("✅ Agent state defined")

✅ Agent state defined


Agent Graph Creation

In [26]:
def create_financial_agent(agent_type: str = "full", with_memory: bool = True):
    """
    Creates a financial research agent with specified configuration.

    Args:
        agent_type: Type of agent to create:
            - "traditional": Simple reactive LLM
            - "basic": Basic goal-oriented agent
            - "full": Full autonomous agent with all constraints
        with_memory: Whether to enable conversation memory

    Returns:
        Compiled LangGraph agent
    """
    # Select system prompt based on agent type
    prompt_map = {
        "traditional": TRADITIONAL_PROMPT,
        "basic": AGENT_CHARTER_BASIC,
        "full": AGENT_CHARTER_FULL
    }

    system_prompt = prompt_map.get(agent_type, AGENT_CHARTER_FULL)

    # Collect all tools
    tools = [ get_stock_price,
    get_stock_history,
    search_financial_news,
    analyze_sentiment ]

    logger.info(f"📦 Creating {agent_type.upper()} agent with {len(tools)} tools")
    logger.info(f"   Tools: {', '.join(t.name for t in tools)}")

    # Initialize model with tools
    model = ChatOpenAI(model="gpt-4o-mini", temperature=0)
    model_with_tools = model.bind_tools(tools)

    # Define agent node
    def agent_node(state: SimpleAgentState) -> dict:
        """Agent node that calls the LLM with system prompt and current state."""
        logger.info("🤖 AGENT NODE: Processing request...")

        # Prepare messages with system prompt
        system_msg = SystemMessage(content=system_prompt)
        messages = [system_msg] + list(state["messages"])

        # Invoke model
        logger.info("   Calling LLM with tools...")
        response = model_with_tools.invoke(messages)

        # Log if agent wants to use tools
        if hasattr(response, 'tool_calls') and response.tool_calls:
            logger.info(f"   ✓ Agent decided to use {len(response.tool_calls)} tool(s)")
            for i, tool_call in enumerate(response.tool_calls, 1):
                logger.info(f"      {i}. {tool_call['name']}")
        else:
            logger.info("   ✓ Agent generated final response (no tools needed)")

        return {"messages": [response]}

    # Define routing function
    def should_continue(state: SimpleAgentState) -> Literal["tools", "end"]:
        """Determines whether to continue to tools or end."""
        last_message = state["messages"][-1]

        if hasattr(last_message, 'tool_calls') and last_message.tool_calls:
            logger.info("🔀 ROUTING: Continuing to TOOLS node")
            return  "tools"

        logger.info("🔀 ROUTING: Ending workflow (final response ready)")
        return  "end"
    # Create workflow
    workflow = StateGraph(SimpleAgentState)

    # Create tool node with logging
    original_tool_node = ToolNode(tools)

    def tool_node_with_logging(state):
        logger.info("🔧 TOOL NODE: Executing tools...")
        result = original_tool_node.invoke(state)
        logger.info(f"   ✓ Tools executed successfully")
        return result

    # Add nodes to graph
    workflow.add_node("agent", agent_node)
    workflow.add_node("tools", tool_node_with_logging)

    # Set entry point
    workflow.set_entry_point("agent")

    # Add conditional edge from agent
    workflow.add_conditional_edges(
        "agent",
        should_continue,
        {
            "tools": "tools",
            "end": END
        }
    )

    # Add edge from tools back to agent
    workflow.add_edge("tools", "agent")

    # Compile graph
    if with_memory:
        logger.info("💾 Enabling conversation memory")
        memory = MemorySaver()
        graph = workflow.compile(checkpointer=memory)
    else:
        logger.info("⚠️  Memory disabled - stateless mode")
        graph = workflow.compile()

    logger.info("✅ Agent created successfully\n")
    return graph

print("✅ Agent creation function defined")

✅ Agent creation function defined


Test 1: Traditional LLM Reactive

In [27]:
print("="*80)
print("TEST 1: Traditional Reactive LLM")
print("="*80 + "\n")

# Create traditional agent
traditional_agent = create_financial_agent(agent_type="traditional", with_memory=False)

# Test query
query = "Tell me about Apple stock"
print(f"Query: {query}\n")
print("-"*80 + "\n")

# Run agent
result = traditional_agent.invoke({"messages": [HumanMessage(content=query)]})

print("\n🤖 TRADITIONAL LLM RESPONSE:")
print("="*80)
print(result["messages"][-1].content)
print("\n" + "="*80)
print("\n❌ Notice: The traditional LLM may ask follow-up questions instead of taking action!")

12:28:47 [INFO] 📦 Creating TRADITIONAL agent with 4 tools
12:28:47 [INFO]    Tools: get_stock_price, get_stock_history, search_financial_news, analyze_sentiment
12:28:47 [INFO] ⚠️  Memory disabled - stateless mode
12:28:47 [INFO] ✅ Agent created successfully

12:28:47 [INFO] 🤖 AGENT NODE: Processing request...
12:28:47 [INFO]    Calling LLM with tools...


TEST 1: Traditional Reactive LLM

Query: Tell me about Apple stock

--------------------------------------------------------------------------------



12:28:48 [INFO] HTTP Request: POST https://aibe.mygreatlearning.com/openai/v1/chat/completions "HTTP/1.1 200 OK"
12:28:48 [INFO]    ✓ Agent decided to use 1 tool(s)
12:28:48 [INFO]       1. get_stock_price
12:28:48 [INFO] 🔀 ROUTING: Continuing to TOOLS node
12:28:48 [INFO] 🔧 TOOL NODE: Executing tools...
12:28:48 [INFO]    ✓ Tools executed successfully
12:28:48 [INFO] 🤖 AGENT NODE: Processing request...
12:28:48 [INFO]    Calling LLM with tools...
12:28:51 [INFO] HTTP Request: POST https://aibe.mygreatlearning.com/openai/v1/chat/completions "HTTP/1.1 200 OK"
12:28:51 [INFO]    ✓ Agent generated final response (no tools needed)
12:28:51 [INFO] 🔀 ROUTING: Ending workflow (final response ready)



🤖 TRADITIONAL LLM RESPONSE:
As of the latest data, here is the information about Apple Inc. (AAPL) stock:

- **Current Price**: $261.45 USD
- **Day's High**: $262.23 USD
- **Day's Low**: $255.54 USD
- **Volume**: 20,086,962 shares traded
- **Market Capitalization**: $3.84 trillion USD

If you need more specific information or historical data, feel free to ask!


❌ Notice: The traditional LLM may ask follow-up questions instead of taking action!


Test 2: Basic Autonomous Agent (Goal-Oriented)

In [28]:
print("="*80)
print("TEST 2: Basic Autonomous Agent (Goal-Oriented)")
print("="*80 + "\n")

# Create basic agent
basic_agent = create_financial_agent(agent_type="basic", with_memory=False)

# Test query
query = "Tell me about Apple stock"
print(f"Query: {query}\n")
print("-"*80 + "\n")

# Run agent
result = basic_agent.invoke({"messages": [HumanMessage(content=query)]})

print("\n🤖 BASIC AGENT RESPONSE:")
print("="*80)
print(result["messages"][-1].content)
print("\n" + "="*80)
print("\n✅ Notice: The agent takes initiative and uses tools to gather data!")

12:29:37 [INFO] 📦 Creating BASIC agent with 4 tools
12:29:37 [INFO]    Tools: get_stock_price, get_stock_history, search_financial_news, analyze_sentiment
12:29:37 [INFO] ⚠️  Memory disabled - stateless mode
12:29:37 [INFO] ✅ Agent created successfully

12:29:37 [INFO] 🤖 AGENT NODE: Processing request...
12:29:37 [INFO]    Calling LLM with tools...


TEST 2: Basic Autonomous Agent (Goal-Oriented)

Query: Tell me about Apple stock

--------------------------------------------------------------------------------



12:29:39 [INFO] HTTP Request: POST https://aibe.mygreatlearning.com/openai/v1/chat/completions "HTTP/1.1 200 OK"
12:29:39 [INFO]    ✓ Agent decided to use 3 tool(s)
12:29:39 [INFO]       1. get_stock_price
12:29:39 [INFO]       2. get_stock_history
12:29:39 [INFO]       3. search_financial_news
12:29:39 [INFO] 🔀 ROUTING: Continuing to TOOLS node
12:29:39 [INFO] 🔧 TOOL NODE: Executing tools...
12:29:41 [INFO]    ✓ Tools executed successfully
12:29:41 [INFO] 🤖 AGENT NODE: Processing request...
12:29:41 [INFO]    Calling LLM with tools...
12:29:51 [INFO] HTTP Request: POST https://aibe.mygreatlearning.com/openai/v1/chat/completions "HTTP/1.1 200 OK"
12:29:51 [INFO]    ✓ Agent generated final response (no tools needed)
12:29:51 [INFO] 🔀 ROUTING: Ending workflow (final response ready)



🤖 BASIC AGENT RESPONSE:
### Apple Inc. (AAPL) Financial Analysis Report

#### 1. Current Stock Price and 3-Year Performance Trends
- **Current Stock Price**: $261.84 USD
- **Market Capitalization**: $3.85 trillion
- **Day's Range**: $255.54 - $262.23
- **Volume**: 20,146,506 shares traded

**3-Year Performance**:
- **Start Price (Feb 17, 2023)**: $150.37
- **End Price (Feb 17, 2026)**: $261.84
- **Return Percentage**: 74.13%
- **High**: $288.35
- **Low**: $141.85
- **Average Volume**: 55,950,742 shares

#### 2. Recent Financial News and Market Sentiment
Recent news articles indicate a mixed sentiment surrounding Apple:
- **AI Investments Scrutiny**: Apple is facing scrutiny over its AI investments and the performance of its news service. Analysts are questioning the company's valuation amidst broader tech market challenges. [Read more here](https://finance.yahoo.com/quote/AAPL/news/).
- **Selloff Concerns**: A recent selloff in Apple stock has been deemed "unwarranted" by Wedbush anal

Test 3: Full Autonomous Agent (All Constraints)

In [29]:
print("="*80)
print("TEST 3: Full Autonomous Agent (With All Constraints)")
print("="*80 + "\n")

# Create full agent
full_agent = create_financial_agent(agent_type="full", with_memory=True)

# Test query
query = "Provide a comprehensive investment analysis for Microsoft (MSFT) including 3-year performance and AI research activity"
print(f"Query: {query}\n")
print("-"*80 + "\n")

# Run agent with memory
config = {"configurable": {"thread_id": "test_session_1"}}
result = full_agent.invoke(
    {"messages": [HumanMessage(content=query)]},
    config=config
)

print("\n🤖 FULL AGENT RESPONSE:")
print("="*80)
print(result["messages"][-1].content)
print("\n" + "="*80)
print("\n✅ Notice: The agent provides comprehensive analysis with:")
print("   • Source citations")
print("   • Multiple tool usage")
print("   • Risk assessment")
print("   • Clear recommendation with confidence")
print("   • Data gap acknowledgment")

12:30:32 [INFO] 📦 Creating FULL agent with 4 tools
12:30:32 [INFO]    Tools: get_stock_price, get_stock_history, search_financial_news, analyze_sentiment
12:30:32 [INFO] 💾 Enabling conversation memory
12:30:32 [INFO] ✅ Agent created successfully

12:30:32 [INFO] 🤖 AGENT NODE: Processing request...
12:30:32 [INFO]    Calling LLM with tools...


TEST 3: Full Autonomous Agent (With All Constraints)

Query: Provide a comprehensive investment analysis for Microsoft (MSFT) including 3-year performance and AI research activity

--------------------------------------------------------------------------------



12:30:35 [INFO] HTTP Request: POST https://aibe.mygreatlearning.com/openai/v1/chat/completions "HTTP/1.1 200 OK"
12:30:35 [INFO]    ✓ Agent decided to use 3 tool(s)
12:30:35 [INFO]       1. get_stock_price
12:30:35 [INFO]       2. get_stock_history
12:30:35 [INFO]       3. search_financial_news
12:30:35 [INFO] 🔀 ROUTING: Continuing to TOOLS node
12:30:35 [INFO] 🔧 TOOL NODE: Executing tools...
12:30:37 [INFO]    ✓ Tools executed successfully
12:30:37 [INFO] 🤖 AGENT NODE: Processing request...
12:30:37 [INFO]    Calling LLM with tools...
12:30:53 [INFO] HTTP Request: POST https://aibe.mygreatlearning.com/openai/v1/chat/completions "HTTP/1.1 200 OK"
12:30:53 [INFO]    ✓ Agent generated final response (no tools needed)
12:30:53 [INFO] 🔀 ROUTING: Ending workflow (final response ready)



🤖 FULL AGENT RESPONSE:
### Investment Analysis Report for Microsoft Corporation (MSFT)

#### Executive Summary
Microsoft Corporation (MSFT) has demonstrated strong financial performance over the past three years, with a notable return of 57.39%. The company is heavily investing in AI research and development, positioning itself as a leader in the AI sector. Recent news indicates a positive market sentiment surrounding Microsoft's AI initiatives, which are expected to drive future growth. 

#### Financial Metrics
- **Current Stock Price**: $397.66 [Source: get_stock_price, 2026-02-17 12:30]
- **Market Capitalization**: $2.96 trillion [Source: get_stock_price, 2026-02-17 12:30]
- **3-Year Performance**:
  - **Start Price (Feb 17, 2023)**: $252.67
  - **End Price (Feb 17, 2026)**: $397.66
  - **Return Percentage**: 57.39%
  - **High**: $553.50
  - **Low**: $240.48
  - **Average Volume**: 23,603,118 shares [Source: get_stock_history, 2026-02-17]

#### Market Sentiment
Recent news articles

Simulating Tool Failures

In [30]:
# Create a modified version of get_stock_price that always fails
@tool
def get_stock_price_failing(ticker: str) -> Dict:
    """
    [SIMULATED FAILURE] Returns the current stock price - but fails for testing.

    Args:
        ticker: Stock ticker symbol (e.g., 'AAPL', 'MSFT', 'GOOGL')

    Returns:
        dict: Error response
    """
    return {
        'ticker': ticker.upper(),
        'status': 'error',
        'error': 'API connection timeout - service temporarily unavailable'
    }

print("⚠️  Created failing version of get_stock_price for testing")
print("   This will help us observe the agent's error handling behavior")

⚠️  Created failing version of get_stock_price for testing
   This will help us observe the agent's error handling behavior


Test Error Handling

In [32]:
def create_agent_with_failing_tool():
    """
    Creates an agent where one tool (get_stock_price) always fails.
    This tests the agent's reactivity and error handling.
    """
    system_prompt = AGENT_CHARTER_FULL

    # Use failing tool instead of working one
    tools = [get_stock_price_failing, get_stock_history, search_financial_news, analyze_sentiment]

    logger.info(f"📦 Creating agent with FAILING stock price tool (for testing)")

    model = ChatOpenAI(
        model="gpt-4o-mini",
        temperature=0,
        openai_api_key=os.environ.get("OPENAI_API_KEY"),
        openai_api_base=os.environ.get("OPENAI_API_BASE")
    )
    model_with_tools = model.bind_tools(tools)

    def agent_node(state: SimpleAgentState) -> dict:
        system_msg = SystemMessage(content=system_prompt)
        messages = [system_msg] + list(state["messages"])
        response = model_with_tools.invoke(messages)
        return {"messages": [response]}

    def should_continue(state: SimpleAgentState) -> Literal["tools", "end"]:
        last_message = state["messages"][-1]
        if hasattr(last_message, 'tool_calls') and last_message.tool_calls:
            return "tools"
        return "end"

    workflow = StateGraph(SimpleAgentState)
    workflow.add_node("agent", agent_node)
    workflow.add_node("tools", ToolNode(tools))
    workflow.set_entry_point("agent")
    workflow.add_conditional_edges("agent", should_continue, {"tools": "tools", "end": END})
    workflow.add_edge("tools", "agent")

    return workflow.compile()

print("✅ Agent with failing tool creation function defined")

✅ Agent with failing tool creation function defined


Test Own Query

In [33]:
# Create your agent (choose type: "traditional", "basic", or "full")
my_agent = create_financial_agent("traditional", with_memory=True)

# Configure memory
config = {"configurable": {"thread_id": "my_test_session"}}

# Test with your own query
# Try these examples or create your own:
# - "Analyze Tesla stock"
# - "Compare Microsoft and Google AI initiatives"
# - "What are the risks in investing in NVIDIA?"
# - "Tell me about Amazon's financial performance"

YOUR_QUERY =  "Analyze Tesla stock"

print("="*80)
print("YOUR CUSTOM QUERY TEST")
print("="*80 + "\n")
print(f"Query: {YOUR_QUERY}\n")
print("-"*80 + "\n")

result = my_agent.invoke(
    {"messages": [HumanMessage(content=YOUR_QUERY)]},
    config=config
)

print("\n🤖 AGENT RESPONSE:")
print("="*80)
print(result["messages"][-1].content)
print("\n" + "="*80)

13:18:13 [INFO] 📦 Creating TRADITIONAL agent with 4 tools
13:18:13 [INFO]    Tools: get_stock_price, get_stock_history, search_financial_news, analyze_sentiment
13:18:13 [INFO] 💾 Enabling conversation memory
13:18:13 [INFO] ✅ Agent created successfully

13:18:13 [INFO] 🤖 AGENT NODE: Processing request...
13:18:13 [INFO]    Calling LLM with tools...


YOUR CUSTOM QUERY TEST

Query: Analyze Tesla stock

--------------------------------------------------------------------------------



13:18:16 [INFO] HTTP Request: POST https://aibe.mygreatlearning.com/openai/v1/chat/completions "HTTP/1.1 200 OK"
13:18:16 [INFO]    ✓ Agent decided to use 2 tool(s)
13:18:16 [INFO]       1. get_stock_price
13:18:16 [INFO]       2. get_stock_history
13:18:16 [INFO] 🔀 ROUTING: Continuing to TOOLS node
13:18:16 [INFO] 🔧 TOOL NODE: Executing tools...
13:18:16 [INFO]    ✓ Tools executed successfully
13:18:16 [INFO] 🤖 AGENT NODE: Processing request...
13:18:16 [INFO]    Calling LLM with tools...
13:18:22 [INFO] HTTP Request: POST https://aibe.mygreatlearning.com/openai/v1/chat/completions "HTTP/1.1 200 OK"
13:18:22 [INFO]    ✓ Agent generated final response (no tools needed)
13:18:22 [INFO] 🔀 ROUTING: Ending workflow (final response ready)



🤖 AGENT RESPONSE:
### Tesla Stock Analysis

#### Current Stock Information
- **Ticker**: TSLA
- **Current Price**: $405.84 USD
- **Day's High**: $413.72 USD
- **Day's Low**: $400.51 USD
- **Volume**: 38,076,693 shares
- **Market Cap**: $1.52 trillion USD

#### Historical Performance (Last 1 Year)
- **Start Date**: February 18, 2025
- **End Date**: February 17, 2026
- **Start Price**: $354.11 USD
- **End Price**: $405.84 USD
- **Return Percentage**: 14.61%
- **High Price**: $498.83 USD
- **Low Price**: $214.25 USD
- **Average Volume**: 95,724,172 shares
- **Data Points**: 251

### Summary
Tesla's stock has shown a positive return of 14.61% over the past year, with significant fluctuations in price, reaching a high of $498.83 and a low of $214.25. The current trading volume indicates active trading, and the market capitalization reflects its status as a major player in the automotive and technology sectors.



# RAG Implementation

In [34]:
# Additional imports for RAG implementation
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import PyPDFDirectoryLoader
from langchain_community.vectorstores import Chroma
from langchain_openai import OpenAIEmbeddings

print("✅ RAG libraries imported successfully!")
print("   • RecursiveCharacterTextSplitter - For document chunking")
print("   • PyPDFDirectoryLoader - For loading PDF documents")
print("   • Chroma - Vector database for semantic search")
print("   • OpenAIEmbeddings - For creating embeddings")

✅ RAG libraries imported successfully!
   • RecursiveCharacterTextSplitter - For document chunking
   • PyPDFDirectoryLoader - For loading PDF documents
   • Chroma - Vector database for semantic search
   • OpenAIEmbeddings - For creating embeddings


### Load AI Initiatives 

In [36]:

# Path of all AI Initiative Documents
ai_initiative_pdf_paths = [f"/content/Companies-AI-Initiatives/{file}" for file in os.listdir("/Users/anshimittal/Downloads/Companies-AI-Initiatives/data/raw_pdfs")]
ai_initiative_pdf_paths

['/content/Companies-AI-Initiatives/Companies-AI-Initiatives']

In [37]:
from langchain_community.document_loaders import PyPDFDirectoryLoader 

loader = PyPDFDirectoryLoader(path = "/Users/anshimittal/Downloads/Companies-AI-Initiatives/data/raw_pdfs") # Creating an PDF loader object

#### Split Documents into Chunks

In [38]:
# Chunking the data
from langchain_text_splitters import RecursiveCharacterTextSplitter # type: ignore

# Define text splitter with tiktoken encoding
text_splitter = RecursiveCharacterTextSplitter.from_tiktoken_encoder(
    encoding_name="cl100k_base",
    chunk_size=1000,
    chunk_overlap=200
)

print("✅ Text splitter configured:")
print(f"   • Encoding: cl100k_base (OpenAI)")
print(f"   • Chunk size: 1000 tokens")
print(f"   • Chunk overlap: 200 tokens")
print(f"   • Strategy: Recursive character splitting")

# Load and split documents
print("\n📄 Loading and splitting PDF documents...")
documents = loader.load()
ai_initiative_chunks = text_splitter.split_documents(documents)


print(f"✅ Documents processed successfully!")
print(f"   • Total chunks created: {len(ai_initiative_chunks)}")
print(f"   • Average chunk size: ~1000 tokens")

# Show a sample chunk
if ai_initiative_chunks:
    print(f"\n📋 Sample chunk preview:")
    print(f"   Source: {ai_initiative_chunks[0].metadata.get('source', 'Unknown')}")
    print(f"   Content preview: {ai_initiative_chunks[0].page_content[:200]}...")

✅ Text splitter configured:
   • Encoding: cl100k_base (OpenAI)
   • Chunk size: 1000 tokens
   • Chunk overlap: 200 tokens
   • Strategy: Recursive character splitting

📄 Loading and splitting PDF documents...
✅ Documents processed successfully!
   • Total chunks created: 62
   • Average chunk size: ~1000 tokens

📋 Sample chunk preview:
   Source: /Users/anshimittal/Downloads/Companies-AI-Initiatives/data/raw_pdfs/Companies-AI-Initiatives/IBM.pdf
   Content preview: IBM,  a  global  technology  leader,  leverages  artificial  intelligence  to  transform  enterprise  
operations
 
and
 
drive
 
innovation
 
across
 
industries.
 
Through
 
platforms
 
like
 
Watso...


#### Create Vector Store with Embeddings

In [39]:
from langchain_openai import OpenAIEmbeddings
from langchain_community.vectorstores import Chroma


# Initialize OpenAI embedding model (text-embedding-ada-002)
embedding_model = OpenAIEmbeddings(model="text-embedding-ada-002")

print("✅ Embedding model initialized: text-embedding-ada-002")
print("   • Embedding dimension: 1536")
print("   • Use case: Semantic similarity search")

# Create vector store from documents
print("\n🔄 Creating vector store (this may take a moment)...")
print("   Generating embeddings for all chunks...")

vectorstore = Chroma.from_documents(
    documents=ai_initiative_chunks,
    embedding=embedding_model,
    collection_name="AI_Initiatives"
)

print(f"\n✅ Vector store created successfully!")
print(f"   • Collection name: AI_Initiatives")
print(f"   • Total vectors: {len(ai_initiative_chunks)}")
print(f"   • Database: ChromaDB (in-memory)")

# Create retriever for similarity search which fetches 10 relevant chunks
retriever = vectorstore.as_retriever(search_kwargs={"k": 10})

print(f"\n✅ Retriever configured:")
print(f"   • Search type: Similarity")
print(f"   • Top-k results: 10")
print(f"   • Ready for queries!")

✅ Embedding model initialized: text-embedding-ada-002
   • Embedding dimension: 1536
   • Use case: Semantic similarity search

🔄 Creating vector store (this may take a moment)...
   Generating embeddings for all chunks...


13:38:41 [INFO] Anonymized telemetry enabled. See                     https://docs.trychroma.com/telemetry for more information.
13:38:50 [INFO] HTTP Request: POST https://aibe.mygreatlearning.com/openai/v1/embeddings "HTTP/1.1 200 OK"



✅ Vector store created successfully!
   • Collection name: AI_Initiatives
   • Total vectors: 62
   • Database: ChromaDB (in-memory)

✅ Retriever configured:
   • Search type: Similarity
   • Top-k results: 10
   • Ready for queries!


#### Test RAG Retrieval (Manual)

In [40]:
# Test query
test_query = "What AI projects is Microsoft working on?"

print(f"🔍 Test Query: {test_query}")
print("="*80)

# Retrieve relevant documents
relevant_docs = retriever.get_relevant_documents(test_query)

print(f"\n✅ Retrieved {len(relevant_docs)} relevant chunks:\n")

# Display top 3 results
for i, doc in enumerate(relevant_docs[:3], 1):
    print(f"📄 Result {i}:")
    print(f"   Source: {doc.metadata.get('source', 'Unknown')}")
    print(f"   Page: {doc.metadata.get('page', 'Unknown')}")
    print(f"   Content: {doc.page_content[:300]}...")
    print("-"*80 + "\n")

🔍 Test Query: What AI projects is Microsoft working on?


/var/folders/dc/flyc_kxs2d554dt8_szth1_40000gn/T/ipykernel_9329/3104663532.py:8: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 1.0. Use :meth:`~invoke` instead.
  relevant_docs = retriever.get_relevant_documents(test_query)
13:39:45 [INFO] HTTP Request: POST https://aibe.mygreatlearning.com/openai/v1/embeddings "HTTP/1.1 200 OK"



✅ Retrieved 10 relevant chunks:

📄 Result 1:
   Source: /Users/anshimittal/Downloads/Companies-AI-Initiatives/data/raw_pdfs/Companies-AI-Initiatives/MSFT.pdf
   Page: 0
   Content: Microsoft,  founded  in  1975,  is  a  global  technology  leader  specializing  in  software,  cloud  
computing,
 
AI,
 
and
 
enterprise
 
solutions.
 
The
 
company
 
has
 
aggressively
 
expanded
 
its
 
AI
 
capabilities
 
through
 
initiatives
 
such
 
as
 
Azure
 
AI,
 
OpenAI
 
partnership
...
--------------------------------------------------------------------------------

📄 Result 2:
   Source: /Users/anshimittal/Downloads/Companies-AI-Initiatives/data/raw_pdfs/Companies-AI-Initiatives/MSFT.pdf
   Page: 3
   Content: ●  Enterprise  Enablement :  Provides  organizations  with  tools  to  integrate  AI  into  their  
operations,
 
improving
 
efficiency,
 
decision-making,
 
and
 
innovation.
 ●  Ecosystem  Growth :  Strengthens  the  Microsoft  ecosystem  by  integrating  Foundry  Labs  
with
 
Az

#### Implement query_private_database tool

In [41]:
@tool
def query_private_database(query: str) -> str:
    """
    Query the private database of analyst reports and AI initiative documents.

    This RAG-powered tool searches through internal company documents about AI initiatives,
    research projects, innovation areas, and strategic technology investments. Use this tool
    when you need information about:
    - Company AI research projects and initiatives
    - AI innovation areas and focus
    - Technology roadmaps and future plans
    - Specific AI project timelines and details

    Args:
        query: Natural language query about company AI initiatives
               (e.g., "What AI projects is Microsoft working on?",
                      "NVIDIA's AI research areas",
                      "Amazon's AI timeline")

    Returns:
        str: Detailed answer based on private analyst reports with source citations

    Example:
        >>> result = query_private_database("What are Google's latest AI initiatives?")
        >>> print(result)
    """
    try:
        # System message for RAG Q&A
        qna_system_message = """You are an assistant specialized in reviewing AI initiatives of companies and providing accurate answers based on the provided context.

User input will include all the context you need to answer their question.
This context will always begin with the token: ###Context.
The context contains references to specific AI initiatives, projects, or programs of companies relevant to the user's query.

User questions will begin with the token: ###Question.

Answer only using the context provided. Do not add external information or mention the context in your answer.
Always cite which company the information comes from.
If the answer cannot be found in the context, respond with "I don't know - this information is not available in our analyst reports."
"""

        # User message template
        qna_user_message_template = """###Context
Here are some documents that are relevant to the question mentioned below.
{context}

###Question
{question}
"""

        # Retrieve relevant document chunks
        relevant_document_chunks = retriever.get_relevant_documents(query)
        context_list = [d.page_content for d in relevant_document_chunks]
        context_for_query = ". ".join(context_list)

        # Build the full prompt
        formatted_prompt = f"""[INST]{qna_system_message}

                {'user'}: {qna_user_message_template.format(context=context_for_query, question=query)}
                [/INST]"""

        # Query the LLM
        model = ChatOpenAI(model="gpt-4o-mini", temperature=0)

        response = model.invoke(formatted_prompt)

        return response.content

    except Exception as e:
        return f"Error querying private database: {str(e)}"

print("✅ Tool 5: query_private_database() - Defined")
print("   Purpose: Access private analyst reports via RAG")
print("   Data Source: ChromaDB vector store")
print("   Powered by: OpenAI embeddings + LLM generation")

✅ Tool 5: query_private_database() - Defined
   Purpose: Access private analyst reports via RAG
   Data Source: ChromaDB vector store
   Powered by: OpenAI embeddings + LLM generation


### Test the RAG Tool

In [42]:
# Test the query_private_database tool
test_queries = [
    "What AI projects is Microsoft working on?",
    "What are NVIDIA's AI research areas?",
    "Tell me about Google's AI initiatives"
]

for test_query in test_queries:
    print("="*80)
    print(f"🔍 Query: {test_query}\n")
    result = query_private_database.invoke({"query": test_query})
    print(f"📄 Answer:\n{result}")
    print("\n")

🔍 Query: What AI projects is Microsoft working on?



13:42:15 [INFO] HTTP Request: POST https://aibe.mygreatlearning.com/openai/v1/embeddings "HTTP/1.1 200 OK"
13:42:21 [INFO] HTTP Request: POST https://aibe.mygreatlearning.com/openai/v1/chat/completions "HTTP/1.1 200 OK"


📄 Answer:
Microsoft is working on several AI projects, including:

1. **Azure AI Foundry Labs**: An experimental AI platform designed to accelerate the translation of advanced AI research into real-world applications. It serves as a collaborative hub for developers, startups, enterprises, and Microsoft Research teams, offering early access to cutting-edge AI models, tools, and frameworks.

2. **Microsoft 365 Copilot**: An AI-powered productivity assistant embedded across Microsoft 365 applications such as Word, Excel, PowerPoint, Outlook, and Teams. It leverages large language models and Microsoft Graph to provide contextual assistance, automate repetitive tasks, and enhance productivity.

These initiatives aim to enhance productivity, streamline workflows, and drive digital transformation while focusing on responsible AI deployment and compliance with ethical standards.


🔍 Query: What are NVIDIA's AI research areas?



13:42:22 [INFO] HTTP Request: POST https://aibe.mygreatlearning.com/openai/v1/embeddings "HTTP/1.1 200 OK"
13:42:24 [INFO] HTTP Request: POST https://aibe.mygreatlearning.com/openai/v1/chat/completions "HTTP/1.1 200 OK"


📄 Answer:
NVIDIA's AI research areas include deep learning, generative AI, digital twins, and industrial automation. The company leverages its GPUs, specialized chips, and software ecosystems like CUDA and Omniverse to drive innovation across various industries, including healthcare and manufacturing.


🔍 Query: Tell me about Google's AI initiatives



13:42:25 [INFO] HTTP Request: POST https://aibe.mygreatlearning.com/openai/v1/embeddings "HTTP/1.1 200 OK"
13:42:33 [INFO] HTTP Request: POST https://aibe.mygreatlearning.com/openai/v1/chat/completions "HTTP/1.1 200 OK"


📄 Answer:
Google has positioned itself as a leader in artificial intelligence (AI) through several key initiatives:

1. **Google Brain and DeepMind**: These are foundational research groups that focus on advancing AI technologies and applications.

2. **Gemini**: This is Google/DeepMind’s flagship family of multimodal foundation models designed to enhance reasoning, coding, and generative capabilities across text, images, and code. Gemini integrates with various Google products, including Bard, Chrome, and Google Workspace, and comes in multiple tiers optimized for different capabilities.

3. **Project Astra**: An advanced AI research prototype aimed at creating a universal AI assistant that integrates seamlessly into daily life. Astra combines multimodal capabilities, processing audio, video, and contextual data to provide real-time assistance across devices, including smartphones and smart glasses.

4. **AI Ecosystem**: Google’s AI ecosystem spans natural language processing, compute

### Updated Agent Charter

In [43]:
AGENT_CHARTER_WITH_RAG = """You are an autonomous Financial Research Analyst Agent specializing in AI sector investments.

════════════════════════════════════════════════════════════════════════════════
PRIMARY MISSION
════════════════════════════════════════════════════════════════════════════════

Analyze public companies (especially AI-focused) to generate comprehensive, real-time
investment research briefings that provide insights beyond simple data lookup.

TARGET OUTPUT:
A structured report covering:
• Financial Health: Stock performance, 3-year trends, key metrics
• Market Sentiment: News analysis with sentiment scores
• AI Research Activity: Current AI projects and innovations (using private database)
• Risk Assessment: Key risks and opportunities
• Investment Recommendation: Data-driven rating with confidence level

════════════════════════════════════════════════════════════════════════════════
AVAILABLE TOOLS
════════════════════════════════════════════════════════════════════════════════

Stock Data Tools:
• get_stock_price(ticker) - Current price, volume, market cap
• get_stock_history(ticker, period) - Historical data (use '3y' for 3-year analysis)

News & Sentiment Tools:
• search_financial_news(query) - Real-time financial news search
• analyze_sentiment(text) - Sentiment analysis with score

Private Data Tools (NEW!):
• query_private_database(query) - Query internal analyst reports about AI initiatives
  Use this for: AI projects, research areas, innovation timelines, technology roadmaps

════════════════════════════════════════════════════════════════════════════════
PROACTIVE BEHAVIOR - Take Initiative
════════════════════════════════════════════════════════════════════════════════

✓ ALWAYS gather comprehensive data, not just what's explicitly requested
✓ ALWAYS check 3-year historical performance, not just current price
✓ ALWAYS analyze recent news sentiment, even if not asked
✓ ALWAYS query private database for AI research activity
✓ ALWAYS identify risks proactively, don't wait to be asked
✓ ALWAYS make a clear recommendation with confidence level

✗ NEVER stop at surface-level data
✗ NEVER provide analysis without supporting evidence
✗ NEVER ignore warning signs in the data
✗ NEVER skip the AI research check

════════════════════════════════════════════════════════════════════════════════
REACTIVE BEHAVIOR - Error Handling & Adaptability
════════════════════════════════════════════════════════════════════════════════

When Tools Fail:
• If a tool returns an error, IMMEDIATELY try an alternative approach
• If stock data fails, explain the limitation and use news/company info instead
• If news search fails, note this gap and continue with available data
• If private database query fails, note the limitation in your report
• NEVER stop your analysis due to a single tool failure
• Log all errors but maintain momentum toward your goal

When Data is Missing:
• If you cannot get 3-year data, use whatever period is available and note it
• If sentiment analysis fails, make qualitative assessment from news titles
• If news is sparse, note this as a finding (low media coverage = risk/opportunity?)
• If AI research data is unavailable, explicitly state this gap
• ALWAYS work with what you have, document what you don't have

════════════════════════════════════════════════════════════════════════════════
AUTONOMOUS BEHAVIOR - Independence & Judgment
════════════════════════════════════════════════════════════════════════════════

Data Gaps & Transparency:
• If you encounter missing data, EXPLICITLY state the gap in your report
• Explain the impact of missing data on your analysis confidence
• NEVER pretend to have data you don't have

Source Citation (MANDATORY):
• You MUST cite the source for every factual claim
• Include timestamps for time-sensitive data (stock prices, news)
• For private database queries, cite as [Source: Private Analyst Reports]
• For news articles, you MUST include the article URL as a clickable link
• Format: [Source: tool_name, timestamp] or [Source: Article Title (URL)]

Example:
✓ "AAPL is trading at $178.45 [Source: get_stock_price, 2024-10-30 13:30]"
✓ "Recent article: 'Apple announces new AI chip' [Source: TechCrunch (https://techcrunch.com/...)]"
✓ "Recent news shows positive sentiment (score: 0.75) [Source: analyze_sentiment]"
✓ "Microsoft is working on Azure AI integration [Source: Private Analyst Reports]"
✗ "The stock is doing well" (no source, no metrics)
✗ "Recent positive news about Apple" (no article link or title)

IMPORTANT for News Articles:
• When citing news from search_financial_news, ALWAYS extract and include:
  - Article title
  - Article URL (as a clickable link)
• Format: "According to '[Article Title]' [Source: Publication (URL)]"
• Never cite news without providing the actual article link

Confidence & Nuance:
• Include confidence levels for predictions: High/Medium/Low
• Acknowledge uncertainty: "Data suggests..." vs "Data confirms..."
• Note when analysis is limited by data availability

════════════════════════════════════════════════════════════════════════════════
AI RESEARCH ACTIVITY CHECK (Section 2.2)
════════════════════════════════════════════════════════════════════════════════

For EVERY company analysis, you MUST:
1. Query the private database for AI initiatives
2. Identify if the company is actively engaged in AI research/innovation
3. List the latest 3 areas of AI research or projects (if available)
4. Include project timelines and details (if available)

Example queries to private database:
• "What AI projects is [COMPANY] working on?"
• "What are [COMPANY]'s AI research areas?"
• "[COMPANY] AI initiative timeline"

════════════════════════════════════════════════════════════════════════════════
QUALITY STANDARDS
════════════════════════════════════════════════════════════════════════════════

Every Report Must Include:
1. Executive Summary (2-3 sentences)
2. Financial Metrics (with sources and timestamps)
3. Sentiment Analysis (with scores and article count)
4. AI Research Activity (verified using private database - minimum 3 areas)
5. Risk Factors (minimum 2-3 identified)
6. Recommendation (Buy/Hold/Sell with confidence %)
7. Source Citations (for all claims)
8. Gaps & Limitations (what data was unavailable)

Remember: You are AUTONOMOUS. Take initiative, handle errors gracefully, and
always drive toward your goal of comprehensive investment analysis.
Use ALL available tools, especially the private database for AI research insights.
"""

print("✅ Updated Agent Charter (with RAG tool)")
print("   • Added query_private_database to available tools")
print("   • Added AI Research Activity Check requirements")
print("   • Enhanced quality standards")

✅ Updated Agent Charter (with RAG tool)
   • Added query_private_database to available tools
   • Added AI Research Activity Check requirements
   • Enhanced quality standards


### Create Enhanced Agent with RAG Tool

In [44]:
def create_enhanced_financial_agent(with_rag: bool = True, with_memory: bool = True):
    """
    Creates an enhanced financial research agent with RAG capabilities.

    Args:
        with_rag: Whether to include the query_private_database RAG tool
        with_memory: Whether to enable conversation memory

    Returns:
        Compiled LangGraph agent with RAG capabilities
    """
    # Use RAG-enhanced charter
    system_prompt = AGENT_CHARTER_WITH_RAG

    # Collect all tools (now including RAG tool)
    if with_rag:
        tools = [
            get_stock_price,
            get_stock_history,
            search_financial_news,
            analyze_sentiment,
            query_private_database  # NEW!
        ]
        logger.info(f"📦 Creating ENHANCED agent with {len(tools)} tools (including RAG)")
    else:
        tools = [get_stock_price, get_stock_history, search_financial_news, analyze_sentiment]
        logger.info(f"📦 Creating agent with {len(tools)} tools (no RAG)")

    logger.info(f"   Tools: {', '.join(t.name for t in tools)}")

    # Initialize model with tools
    model = ChatOpenAI(model="gpt-4o-mini", temperature=0)
    model_with_tools = model.bind_tools(tools)

    # Define agent node
    def agent_node(state: SimpleAgentState) -> dict:
        """Agent node that calls the LLM with system prompt and current state."""
        logger.info("🤖 AGENT NODE: Processing request...")

        # Prepare messages with system prompt
        system_msg = SystemMessage(content=system_prompt)
        messages = [system_msg] + list(state["messages"])

        # Invoke model
        logger.info("   Calling LLM with tools...")
        response = model_with_tools.invoke(messages)

        # Log if agent wants to use tools
        if hasattr(response, 'tool_calls') and response.tool_calls:
            logger.info(f"   ✓ Agent decided to use {len(response.tool_calls)} tool(s)")
            for i, tool_call in enumerate(response.tool_calls, 1):
                logger.info(f"      {i}. {tool_call['name']}")
        else:
            logger.info("   ✓ Agent generated final response (no tools needed)")

        return {"messages": [response]}

    # Define routing function
    def should_continue(state: SimpleAgentState) -> Literal["tools", "end"]:
        """Determines whether to continue to tools or end."""
        last_message = state["messages"][-1]

        if hasattr(last_message, 'tool_calls') and last_message.tool_calls:
            logger.info("🔀 ROUTING: Continuing to TOOLS node")
            return "tools"


        logger.info("🔀 ROUTING: Ending workflow (final response ready)")
        return "end"

    # Create workflow
    workflow = StateGraph(SimpleAgentState)

    # Create tool node with logging
    original_tool_node = ToolNode(tools)

    def tool_node_with_logging(state):
        logger.info("🔧 TOOL NODE: Executing tools...")
        result = original_tool_node.invoke(state)
        logger.info(f"   ✓ Tools executed successfully")
        return result

    # Add nodes to graph
    workflow.add_node("agent", agent_node)
    workflow.add_node("tools", tool_node_with_logging)

    # Set entry point
    workflow.set_entry_point("agent")

    # Add conditional edge from agent
    workflow.add_conditional_edges(
        "agent",
        should_continue,
        {
            "tools": "tools",
            "end": END
        }
    )

    # Add edge from tools back to agent
    workflow.add_edge("tools", "agent")

    # Compile graph
    if with_memory:
        logger.info("💾 Enabling conversation memory")
        memory = MemorySaver()
        graph = workflow.compile(checkpointer=memory)
    else:
        logger.info("⚠️  Memory disabled - stateless mode")
        graph = workflow.compile()

    logger.info("✅ Enhanced agent created successfully\n")
    return graph

print("✅ Enhanced agent creation function defined")
print("   • Includes all 5 tools (stock, news, sentiment, RAG)")
print("   • Uses updated charter with AI research requirements")

✅ Enhanced agent creation function defined
   • Includes all 5 tools (stock, news, sentiment, RAG)
   • Uses updated charter with AI research requirements


### Agent with RAG - AI Research Activity Check

In [45]:
print("="*80)
print("TEST 5: Enhanced Agent with RAG - AI Research Activity Check")
print("="*80 + "\n")

# Create enhanced agent with RAG
enhanced_agent = create_enhanced_financial_agent(with_rag=True, with_memory=True)

# Test query that requires AI research information
query = "Provide a comprehensive investment analysis for NVIDIA (NVDA) including their AI research initiatives"
print(f"Query: {query}\n")
print("-"*80 + "\n")

# Run agent with memory
config = {"configurable": {"thread_id": "enhanced_test_1"}}
result = enhanced_agent.invoke(
    {"messages": [HumanMessage(content=query)]},
    config=config
)

print("\n🤖 ENHANCED AGENT RESPONSE (with RAG):")
print("="*80)
print(result["messages"][-1].content)
print("\n" + "="*80)
print("\n✅ Notice: The agent now includes:")
print("   • AI research projects from private analyst reports")
print("   • Specific AI initiative details")
print("   • Integration of financial + AI research data")
print("   • Comprehensive investment recommendation")

13:51:34 [INFO] 📦 Creating ENHANCED agent with 5 tools (including RAG)
13:51:34 [INFO]    Tools: get_stock_price, get_stock_history, search_financial_news, analyze_sentiment, query_private_database
13:51:34 [INFO] 💾 Enabling conversation memory
13:51:34 [INFO] ✅ Enhanced agent created successfully

13:51:34 [INFO] 🤖 AGENT NODE: Processing request...
13:51:34 [INFO]    Calling LLM with tools...


TEST 5: Enhanced Agent with RAG - AI Research Activity Check

Query: Provide a comprehensive investment analysis for NVIDIA (NVDA) including their AI research initiatives

--------------------------------------------------------------------------------



13:51:37 [INFO] HTTP Request: POST https://aibe.mygreatlearning.com/openai/v1/chat/completions "HTTP/1.1 200 OK"
13:51:37 [INFO]    ✓ Agent decided to use 4 tool(s)
13:51:37 [INFO]       1. get_stock_price
13:51:37 [INFO]       2. get_stock_history
13:51:37 [INFO]       3. search_financial_news
13:51:37 [INFO]       4. query_private_database
13:51:37 [INFO] 🔀 ROUTING: Continuing to TOOLS node
13:51:37 [INFO] 🔧 TOOL NODE: Executing tools...
13:51:37 [INFO] HTTP Request: POST https://aibe.mygreatlearning.com/openai/v1/embeddings "HTTP/1.1 200 OK"
13:51:42 [INFO] HTTP Request: POST https://aibe.mygreatlearning.com/openai/v1/chat/completions "HTTP/1.1 200 OK"
13:51:42 [INFO]    ✓ Tools executed successfully
13:51:42 [INFO] 🤖 AGENT NODE: Processing request...
13:51:42 [INFO]    Calling LLM with tools...
13:52:04 [INFO] HTTP Request: POST https://aibe.mygreatlearning.com/openai/v1/chat/completions "HTTP/1.1 200 OK"
13:52:04 [INFO]    ✓ Agent generated final response (no tools needed)
13:52:0


🤖 ENHANCED AGENT RESPONSE (with RAG):
### Comprehensive Investment Analysis for NVIDIA (NVDA)

#### Executive Summary
NVIDIA Corporation (NVDA) has demonstrated exceptional growth over the past three years, with a staggering return of 767.38%. The company's focus on artificial intelligence (AI) and its innovative projects position it as a leader in the tech sector. Recent advancements in AI technology, particularly in local processing and gaming, further enhance its market potential. Given the strong financial health and positive market sentiment, a **Buy** recommendation is warranted with a high confidence level.

---

#### 1. Financial Health
- **Current Stock Price**: $185.33 [Source: get_stock_price, 2026-02-17 13:51]
- **Market Capitalization**: $4.51 trillion
- **3-Year Performance**:
  - **Start Price (Feb 17, 2023)**: $21.37
  - **End Price (Feb 17, 2026)**: $185.33
  - **Return Percentage**: 767.38%
  - **High**: $212.18
  - **Low**: $20.40
  - **Average Volume**: 343,576,905

### Synergistic Tool Usage

In [46]:
print("="*80)
print("TEST 6: Synergistic Tool Usage (News + Sentiment + RAG)")
print("="*80 + "\n")

query = "Analyze Microsoft's position in the AI market. Include recent news sentiment and their strategic AI initiatives."
print(f"Query: {query}\n")
print("-"*80 + "\n")

config = {"configurable": {"thread_id": "synergy_test_1"}}
result = enhanced_agent.invoke(
    {"messages": [HumanMessage(content=query)]},
    config=config
)

print("\n🤖 AGENT RESPONSE (Synergistic Tool Usage):")
print("="*80)
print(result["messages"][-1].content)
print("\n" + "="*80)
print("\n✅ The agent demonstrated synergistic tool usage:")
print("   1. search_financial_news() - Found recent articles")
print("   2. analyze_sentiment() - Analyzed news sentiment")
print("   3. query_private_database() - Retrieved AI initiative details")
print("   4. get_stock_history() - Got financial performance")
print("   5. Synthesized all data into comprehensive report")

13:52:42 [INFO] 🤖 AGENT NODE: Processing request...
13:52:42 [INFO]    Calling LLM with tools...


TEST 6: Synergistic Tool Usage (News + Sentiment + RAG)

Query: Analyze Microsoft's position in the AI market. Include recent news sentiment and their strategic AI initiatives.

--------------------------------------------------------------------------------



13:52:45 [INFO] HTTP Request: POST https://aibe.mygreatlearning.com/openai/v1/chat/completions "HTTP/1.1 200 OK"
13:52:45 [INFO]    ✓ Agent decided to use 4 tool(s)
13:52:45 [INFO]       1. get_stock_price
13:52:45 [INFO]       2. get_stock_history
13:52:45 [INFO]       3. search_financial_news
13:52:45 [INFO]       4. query_private_database
13:52:45 [INFO] 🔀 ROUTING: Continuing to TOOLS node
13:52:45 [INFO] 🔧 TOOL NODE: Executing tools...
13:52:45 [INFO] HTTP Request: POST https://aibe.mygreatlearning.com/openai/v1/embeddings "HTTP/1.1 200 OK"
13:52:49 [INFO] HTTP Request: POST https://aibe.mygreatlearning.com/openai/v1/chat/completions "HTTP/1.1 200 OK"
13:52:49 [INFO]    ✓ Tools executed successfully
13:52:49 [INFO] 🤖 AGENT NODE: Processing request...
13:52:49 [INFO]    Calling LLM with tools...
13:53:07 [INFO] HTTP Request: POST https://aibe.mygreatlearning.com/openai/v1/chat/completions "HTTP/1.1 200 OK"
13:53:07 [INFO]    ✓ Agent generated final response (no tools needed)
13:53:0


🤖 AGENT RESPONSE (Synergistic Tool Usage):
### Microsoft Corporation (MSFT) Investment Analysis

#### Executive Summary
Microsoft Corporation is strategically positioned as a leader in the AI market, leveraging its extensive resources to drive innovation and enhance productivity across various sectors. Recent developments indicate a strong commitment to ethical AI practices and significant investments in AI technologies, particularly in healthcare and environmental sustainability.

---

#### Financial Metrics
- **Current Stock Price**: $397.44 USD [Source: get_stock_price, 2026-02-17 13:52]
- **Market Capitalization**: $2.95 trillion USD
- **3-Year Performance**:
  - **Start Price (Feb 2023)**: $252.67
  - **End Price (Feb 2026)**: $397.44
  - **Return**: 57.3%
  - **High**: $553.50
  - **Low**: $240.48
  - **Average Volume**: 23,608,018 shares [Source: get_stock_history, 2026-02-17]

---

#### Market Sentiment
Recent news articles reflect a predominantly positive sentiment towards Mi

## Investment Recommendation System

### Multi-Company Ranking

In [49]:
print("="*80)
print("TEST 7: Investment Recommendation System - Multi-Company Ranking")
print("="*80 + "\n")

# Define companies to analyze
companies = ["MSFT", "GOOGL", "NVDA", "AMZN", "IBM"]

query = f"""

You are a senior investment analyst. Provide a comparative investment analysis
for the following companies.

For EACH company:
1. Current stock performance
2. 3-year historical return
3. Recent financial news sentiment
4. AI research initiatives and strategic AI positioning
5. Risk factors
6. Long-term growth outlook

Then:
- Rank the companies from strongest to weakest investment opportunity
- Provide justification for the ranking
- Clearly state a top pick and why

Companies to analyze: {', '.join(companies)}

Deliver the output as:
- Individual company summaries
- A ranking table
- Final investment recommendation

"""

print(f"Query: Multi-company investment ranking\n")
print(f"Companies: {', '.join(companies)}\n")
print("-"*80 + "\n")

config = {"configurable": {"thread_id": "ranking_test_1"}}
result = enhanced_agent.invoke(
    {"messages": [HumanMessage(content=query)]},
    config=config
)

print("\n🤖 INVESTMENT RANKING REPORT:")
print("="*80)
print(result["messages"][-1].content)
print("\n" + "="*80)


13:55:28 [INFO] 🤖 AGENT NODE: Processing request...
13:55:28 [INFO]    Calling LLM with tools...


TEST 7: Investment Recommendation System - Multi-Company Ranking

Query: Multi-company investment ranking

Companies: MSFT, GOOGL, NVDA, AMZN, IBM

--------------------------------------------------------------------------------



13:55:36 [INFO] HTTP Request: POST https://aibe.mygreatlearning.com/openai/v1/chat/completions "HTTP/1.1 200 OK"
13:55:36 [INFO]    ✓ Agent decided to use 20 tool(s)
13:55:36 [INFO]       1. get_stock_price
13:55:36 [INFO]       2. get_stock_price
13:55:36 [INFO]       3. get_stock_price
13:55:36 [INFO]       4. get_stock_price
13:55:36 [INFO]       5. get_stock_price
13:55:36 [INFO]       6. get_stock_history
13:55:36 [INFO]       7. get_stock_history
13:55:36 [INFO]       8. get_stock_history
13:55:36 [INFO]       9. get_stock_history
13:55:36 [INFO]       10. get_stock_history
13:55:36 [INFO]       11. search_financial_news
13:55:36 [INFO]       12. search_financial_news
13:55:36 [INFO]       13. search_financial_news
13:55:36 [INFO]       14. search_financial_news
13:55:36 [INFO]       15. search_financial_news
13:55:36 [INFO]       16. query_private_database
13:55:36 [INFO]       17. query_private_database
13:55:36 [INFO]       18. query_private_database
13:55:36 [INFO]       19. 


🤖 INVESTMENT RANKING REPORT:
### Comparative Investment Analysis

#### 1. Microsoft Corporation (MSFT)
- **Current Stock Performance**: $397.40, Market Cap: $2.95 Trillion [Source: get_stock_price, 2026-02-17 13:55]
- **3-Year Historical Return**: 57.28% [Source: get_stock_history, 2026-02-17]
- **Recent Financial News Sentiment**: Positive sentiment (score: 0.82) with articles highlighting advancements in AI, including the launch of Copilot and Azure AI initiatives [Source: Financial News].
- **AI Research Initiatives**:
  - **Azure AI Foundry Labs**: A platform for translating AI research into applications.
  - **Microsoft 365 Copilot**: AI assistant for productivity tools.
  - **GitHub Copilot**: AI-driven coding support tool.
- **Risk Factors**: 
  - Heavy reliance on cloud services.
  - Competition from other tech giants in AI.
- **Long-Term Growth Outlook**: Strong, driven by continuous innovation in AI and cloud services.

#### 2. Alphabet Inc. (GOOGL)
- **Current Stock Perform

### Custom Query

In [50]:
custom_query = "Which company has the most innovative AI research among the analyzed companies? Provide detailed evidence from the private analyst reports."

print("="*80)
print("YOUR CUSTOM QUERY")
print("="*80 + "\n")
print(f"Query: {custom_query}\n")
print("-"*80 + "\n")

config = {"configurable": {"thread_id": "ranking_test_1"}}
result = enhanced_agent.invoke(
    {"messages": [HumanMessage(content=custom_query)]},
    config=config
)

print("\n🤖 AGENT RESPONSE:")
print("="*80)
print(result["messages"][-1].content)
print("\n" + "="*80)

13:57:58 [INFO] 🤖 AGENT NODE: Processing request...
13:57:58 [INFO]    Calling LLM with tools...


YOUR CUSTOM QUERY

Query: Which company has the most innovative AI research among the analyzed companies? Provide detailed evidence from the private analyst reports.

--------------------------------------------------------------------------------



13:58:03 [INFO] HTTP Request: POST https://aibe.mygreatlearning.com/openai/v1/chat/completions "HTTP/1.1 200 OK"
13:58:03 [INFO]    ✓ Agent decided to use 5 tool(s)
13:58:03 [INFO]       1. query_private_database
13:58:03 [INFO]       2. query_private_database
13:58:03 [INFO]       3. query_private_database
13:58:03 [INFO]       4. query_private_database
13:58:03 [INFO]       5. query_private_database
13:58:03 [INFO] 🔀 ROUTING: Continuing to TOOLS node
13:58:03 [INFO] 🔧 TOOL NODE: Executing tools...
13:58:03 [INFO] HTTP Request: POST https://aibe.mygreatlearning.com/openai/v1/embeddings "HTTP/1.1 200 OK"
13:58:03 [INFO] HTTP Request: POST https://aibe.mygreatlearning.com/openai/v1/embeddings "HTTP/1.1 200 OK"
13:58:04 [INFO] HTTP Request: POST https://aibe.mygreatlearning.com/openai/v1/embeddings "HTTP/1.1 200 OK"
13:58:05 [INFO] HTTP Request: POST https://aibe.mygreatlearning.com/openai/v1/embeddings "HTTP/1.1 200 OK"
13:58:07 [INFO] HTTP Request: POST https://aibe.mygreatlearning.com


🤖 AGENT RESPONSE:
### Analysis of AI Research Innovations

#### 1. Microsoft Corporation (MSFT)
- **AI Projects**:
  - **Azure AI Foundry Labs**: An experimental platform that accelerates the translation of advanced AI research into real-world applications, serving as a collaborative hub for developers and researchers.
  - **Microsoft 365 Copilot**: An AI-powered assistant integrated into Microsoft 365 applications, enhancing productivity through contextual assistance and automation.
- **Innovation Focus**: Microsoft emphasizes responsible AI deployment and compliance with ethical standards, aiming to enhance productivity and streamline workflows.

#### 2. Alphabet Inc. (GOOGL)
- **AI Projects**:
  - **Gemini**: A flagship family of multimodal foundation models designed to enhance reasoning, coding, and generative capabilities across various media types, integrated into Google products.
  - **Project Astra**: A prototype for a universal AI assistant that processes audio, video, and co

### Summary and Future Scope

Observations:

1. combines difference sources of data from private RAG database to real-time market value and web sentiment
2. agent demonstrates autonomous tool routing; thus, decides when to call which tool
3. connects financial metrics with strategic AI positioning

Future Scope
1. Could include a composite Investment Score for ranking system
2. add risk modeling and monte carlo simulation
3. convert into web app with interactive dashboard